# ReCCO = Reconstruction Code for Cosmological Observables

The code consists of two main modules that handle the following tasks:

1) $\textbf{spectra.py}$  computes angular power spectra for a variety of cosmological fields on our past lightcone. 

2) $\textbf{estim.py}$ takes input angular power spectra and runs the quadratic estimator machinery: computation of reconstruction noise, biases, principal component analysis, and Gaussian simulations.

## Basic concepts

Mutiple cosmological fields can be defined on our past lightcone and used to construct cosmological observables. A three dimensional field can be integrated or coarse grained along the radial direction of the lightcone in order to generate 1 or multiple two-dimensional fields, each with its own angular power spectrum Cl. 

$\textbf{spectra.py}$ generates Cls for fully integrated and coarse grained fields on our past lightcone. For coarse grained fields, our choice of radial coarse graining for the lightcone consists on separating it into N equal comoving size radial bins.

For example, consider the 3-dimensional radial veloicty defined on our past lightcone. $\textbf{spectra.py}$ separates the lightcone into N redshift bins, averages the radial velocity field inside each bin, and computes the angular auto and cross spectra between bins. The output is a matrix with shape (NL, N, N), where NL is the number of $\ell$ samples. 


## spectra.py

### Base layer computations

$\textbf{spectra.py}$ is divided into two layers of computation. The base layer takes care of computing Cls for cosmological fields that serve as building blocks to compute more complex signals. The output of spectra.py are matrices with with shape (NL, N1,N2). NL is the number of sampled $\ell$ multipoles.

Some base layer fields are computed subdividing the past lightcone into a custom number of redshift bins (Pi-bins):

1) vr (radial component of the velocity field on the lightcone)

2) vt (transverse velocity potential field on the lightcone)

3) g (galaxy number counts on the lightcone)

4) taud (differential optical depth field on the lightcone)

5) ml (moving lens potential field on the lightcone)

6) e (electron density field on the lightcone)

7) m (dark matter density field on the lightcone)

and other base layer fields are fully integrated along the lightcone:

7) CIB (Cosmic Infrarred background at multiple frequencies)

7) tSZ (thermal Sunyaev-Zel'dovich signal at multiple frequencies)

8) isw_lin (linear isw signal)

2) pCMB (primary CMB, both lensed and unlensed)

3) lensing (lensing potential)

One can compute and save the angular power spectra between these observables by running the following in the command line:

$\textbf{python spectra.py -t1 A -t2 B -lmax LMAX}$

where A and B are replaced by any of the fields listed above, and LMAX is replaced by the desired LMAX. For example, if I want to obtain the Cl_g_g up to LMAX of 6144 I need to run:

$\textbf{python spectra.py -t1 g -t2 g -lmax 6144}$

or, if I wanted  Cl_vr_taud up to LMAX of 6144 I need to run:

$\textbf{python spectra.py -t1 vr-t2 taud -lmax 6144}$

$\textbf{NOTE}$: pCMB can only be paired with itself (i.e. A = B = pCMB)

We include a shell script, $\textbf{get_spectra.sh}$, to generate all the necessary Cls for second layer computation. Just run 

$\textbf{bash get_spectra.sh}$

and make sure to modify LMAX accordignly in $\textbf{get_spectra.sh}$ before running.

### Post computation layer

Once the base layer fields are computed, spectra.py allows us to construct several new Cls:

1) $\textbf{kSZ signal}$: The kSZ signal can be constructed from previously computed base layer fields (vr and taud). If I have stored data for Cl_vr_vr, Cl_vr_taud and Cl_taud_taud computed with a number N of bins, then the kSZ signal is computed running:

$\textbf{python spectra.py -postcomp -kSZ N -lmax LMAX}$ 

Example: Let's say I want to calculate the kSZ signal sourced on our past lightcone between redshifts of z_min = 1 and z_max = 2 (set these in config.py) and up to $\ell$ of 6000. First I need to compute Cl_vr_vr, Cl_vr_taud and Cl_taud_taud on the past lightcone, which is subdivided into N_bins (set these in config.py) redshift bins. The higher the choice of N_bins, the more accurate the kSZ will be, but it will take longer. Once z_min, z_max, and N_bins (lets say 64) are set run :

$\textbf{python spectra.py -t1 vr -t2 vr -lmax 6000}$

$\textbf{python spectra.py -t1 taud -t2 taud -lmax 6000}$

$\textbf{python spectra.py -t1 taud -t2 vr -lmax 6000}$

and then run 

$\textbf{python spectra.py -postcomp -kSZ 64 -lmax 6000}$ 

and that will save the kSZ signal. 


2) $\textbf{Moving lens signal}$: The kSZ signal can be constructed from previously computed base layer fields (ml and vt). If I have stored data for Cl_vt_vt, Cl_vt_ml and Cl_ml_ml computed with a number N of bins, then the moving lens signal is computed running:

$\textbf{python spectra.py -postcomp -kSZ N -lmax LMAX}$ 


3)$\textbf{Coarse graining of fields}$: data computed with N_bins = Nfine can be transformed into data with the current N_bins configuration using:

$\textbf{python spectra.py -postcomp -coarse Nfine -lmax LMAX}$ 

assuming Nfine and N_bins actual are both powers of 2 and Nfine > N_bins. 


4) $\textbf{Multi-frequency temperature cleaning}$: The CMB contains multiple frequency dependent foregrounds that 
we might want to clean. Using 

$\textbf{python spectra.py -postcomp -cleanTX A -lmax LMAX}$ 

will return the ILC cleaned CMB temperature (accounting for tSZ,CIB and instrument noise) and additionally , the Cl_TA cleaned spectrum. 

Example: I want to get the frequency cleande Cl_T_T and Cl_T_g spower spectra up to LMAX of 8000. I have to run:

$\textbf{python spectra.py -postcomp -cleanTX g -lmax 8000}$ 
